### Load data
This cell loads data from project data sets
It is marked as #dd-ignore as it will be ignored when importing this notebook to DO for WS to use scenario data management.

In [1]:
#dd-ignore

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_5aea6b8defe6477eb3bb359a3ad797c8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vyscBetMpPlUTXIy_oGwoM89y3Ta7C_-kAOS8SzbMFD7',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_5aea6b8defe6477eb3bb359a3ad797c8.get_object(Bucket='do4wsmarketingcampaignshandson-donotdelete-pr-17nh3lvgn7be59',Key='candidate.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

body = client_5aea6b8defe6477eb3bb359a3ad797c8.get_object(Bucket='do4wsmarketingcampaignshandson-donotdelete-pr-17nh3lvgn7be59',Key='customer.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

body = client_5aea6b8defe6477eb3bb359a3ad797c8.get_object(Bucket='do4wsmarketingcampaignshandson-donotdelete-pr-17nh3lvgn7be59',Key='campaign-1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,id,max customers
0,Home,3
1,Auto,3
2,Travel,3


In [2]:
#dd-ignore
import os, pandas as pd

inputs = {}

inputs['campaign']= df_data_3
inputs['customer'] = df_data_2
inputs['candidate']= df_data_1

inputs['candidate'].head()

outputs = {}

### Extract data from inputs dictionnary
In DO for DSX the data is passed as a dictionnary indexed by table names

In [3]:
import os, pandas as pd

df_campaign = inputs['campaign']
campaigns = df_campaign['id'].values
df_campaign.set_index('id', inplace=True)

df_customer = inputs['customer'] 
customers = df_customer['id'].values
df_customer.set_index('id', inplace=True)

df_candidate = inputs['candidate']
df_candidate.set_index(["Customer","Campaign"], inplace = True)

df_candidate.head()

,,expected value
Customer,Campaign,
Customer 1,Home,70
Customer 2,Home,80
Customer 3,Home,90
Customer 4,Home,50
Customer 5,Home,100


### Import DO package and create a new model

In [4]:
from docplex.mp.model import Model
mdl = Model("MarketingSmall")

### Create decision variables
We will have one "selection" binary decision variable for each pair of cusomer and campaign  

In [5]:
selected = mdl.binary_var_matrix(keys1=customers, keys2=campaigns, name="selected")
df_selected = pd.DataFrame({'selected': selected})
df_selected.index.names=['customer', 'campaign']

### Create and add objective
The objective is to maximize expected revenue

In [6]:
expected_revenue = mdl.sum( selected[customer,campaign] * df_candidate['expected value'][customer,campaign] for customer in customers for campaign in campaigns) 
mdl.add_kpi(expected_revenue, "Expected revenue");
mdl.maximize(expected_revenue);

### Create and add constraints
* For each campaign the number of selected customers is lower than "max customers"

In [7]:
for campaign in campaigns:
    mdl.add_constraint( mdl.sum( selected[customer,campaign] for customer in customers) <= df_campaign['max customers'][campaign])

* For each customer, there is at most one selected campaign

In [8]:
for customer in customers:
    mdl.add_constraint( mdl.sum( selected[customer,campaign] for campaign in campaigns) <= 1)

In [9]:
mdl.print_information()

Model: MarketingSmall
 - number of variables: 30
   - binary=30, integer=0, continuous=0
 - number of constraints: 13
   - linear=13
 - parameters: defaults
 - problem type is: MILP


### Solve the model

In [10]:
assert mdl.solve(), "!!! Solve of the model fails"

In [11]:
mdl.report()

* model MarketingSmall solved with objective = 770
*  KPI: Expected revenue = 770.000


### Extract solution value for selected variables

In [12]:
df_selected = df_selected.selected.apply(lambda v: v.solution_value)
df_selected.head()

customer     campaign
Customer 1   Auto        0.0
             Home        1.0
             Travel      0.0
Customer 10  Auto        0.0
             Home        0.0
Name: selected, dtype: float64

### Build output data frame

In [13]:
df_selected = df_selected.reset_index()
df_candidate = df_candidate.reset_index();
df_selected['expected revenue'] = df_selected.selected * df_candidate['expected value']

df_selected.head()


,customer,campaign,selected,expected revenue
0,Customer 1,Auto,0.0,0.0
1,Customer 1,Home,1.0,80.0
2,Customer 1,Travel,0.0,0.0
3,Customer 10,Auto,0.0,0.0
4,Customer 10,Home,0.0,0.0


And add it to outputs dictionnary so that it is exported to scenario

In [14]:
outputs['selected'] = df_selected